In [457]:
import pandas as pd
from IPython.display import display, clear_output
import time
from datetime import datetime,timedelta
import numpy as np
from scipy.stats import norm

In [303]:
def is_third_friday(d):
    return d.weekday() == 4 and 15 <= d.day <= 21

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

### Connection

In [556]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=16)
ib.client.reqMarketDataType( 3 )

### Read config

In [557]:
cfg = pd.read_json('config.json').set_index('name')
cfg

,CALL_GAP_ALLOW,HARD_CODE_CUM_PROB,INT_RATE,OBSERVED_CUM_PROB_RECENT,OVERRIDE_FORWARD_DATE,PUT_GAP_ALLOW,USE_OVERRIDE_VVIX,VVIX_OVERNIGHT_HARD_CODE,optionsExp,symbol
name,,,,,,,,,,
second,5.0,0.98,0.025,0.9760,1.0,5.0,1.0,87.0,20190521,VXG9
front,5.0,0.85,0.025,0.8617,1.0,5.0,1.0,72.0,20190115,VXF9
sp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190117,SPH9


### Get SPX futures expirations

In [561]:
spxFuts = Future(exchange="GLOBEX",symbol = "SPX",tradingClass = 'SP',secType = "FUT")        
chains = ib.reqContractDetails(spxFuts)
spxFuts=util.df(chains)

SPX=pd.DataFrame(data=spxFuts.contract.apply(lambda x:x.lastTradeDateOrContractMonth).values,index=spxFuts.contract.apply(lambda x:x.localSymbol),columns=['expiration'])
SPX['EXPIRATION']=SPX.apply(lambda x:datetime.strptime(str(x.expiration), '%Y%m%d'),axis=1)
SPX['FWD_START_TIME'] = SPX['EXPIRATION'] + timedelta(minutes=9.5*60)

SPX['IsThirdFriday'] = SPX.EXPIRATION.apply(lambda x: is_third_friday(x))
SPX['WeekDay'] = SPX.EXPIRATION.apply(lambda x: x.weekday())
SPX['IsLastMonthDay'] = SPX.EXPIRATION.apply(lambda x: last_day_of_month(x)==x)

SPX['IsGoodSpx']=SPX.apply(lambda x:not((x.WeekDay==0) | (x.WeekDay==2)) | x.IsLastMonthDay, axis=1)
SPX=SPX[SPX['IsGoodSpx']]
SPX=SPX.sort_values('EXPIRATION')
SPX['FWD_START_TIME'] = SPX['EXPIRATION'] + timedelta(minutes=16*60+15)
SPX.loc[SPX['IsThirdFriday'],'FWD_START_TIME'] = SPX['EXPIRATION'] + timedelta(minutes=9.5*60)
SPX

,expiration,EXPIRATION,FWD_START_TIME,IsThirdFriday,WeekDay,IsLastMonthDay,IsGoodSpx
contract,,,,,,,
SPZ8,20181220,2018-12-20,2018-12-20 16:15:00,False,3,False,True
SPH9,20190314,2019-03-14,2019-03-14 16:15:00,False,3,False,True
SPM9,20190620,2019-06-20,2019-06-20 16:15:00,False,3,False,True
SPU9,20190919,2019-09-19,2019-09-19 16:15:00,False,3,False,True
SPZ9,20191219,2019-12-19,2019-12-19 16:15:00,False,3,False,True
SPH0,20200319,2020-03-19,2020-03-19 16:15:00,False,3,False,True
SPM0,20200618,2020-06-18,2020-06-18 16:15:00,False,3,False,True
SPU0,20200917,2020-09-17,2020-09-17 16:15:00,False,3,False,True
SPZ0,20201217,2020-12-17,2020-12-17 16:15:00,False,3,False,True


### Get VIX futures expirations

In [559]:
vixFut = Future(exchange="CFE",symbol = "VIX",tradingClass = 'VX',secType = "FUT")        
chains = ib.reqContractDetails(vixFut)
vixFut=util.df(chains)
VIX=pd.DataFrame(data=vixFut.contract.apply(lambda x:x.lastTradeDateOrContractMonth).values,index=vixFut.contract.apply(lambda x:x.localSymbol),columns=['expiration'])
VIX['EXPIRATION']=VIX.apply(lambda x:datetime.strptime(str(x.expiration), '%Y%m%d'),axis=1)
VIX['FWD_START_TIME'] = VIX['EXPIRATION'] + timedelta(minutes=9.5*60)
VIX['FWD_END_TIME'] = VIX['FWD_START_TIME'] + timedelta(days = 30)
VIX=VIX.sort_values('EXPIRATION')
VIX

,expiration,EXPIRATION,FWD_START_TIME,FWD_END_TIME
contract,,,,
VXF9,20190116,2019-01-16,2019-01-16 09:30:00,2019-02-15 09:30:00
VXG9,20190213,2019-02-13,2019-02-13 09:30:00,2019-03-15 09:30:00
VXH9,20190319,2019-03-19,2019-03-19 09:30:00,2019-04-18 09:30:00
VXJ9,20190417,2019-04-17,2019-04-17 09:30:00,2019-05-17 09:30:00
VXK9,20190522,2019-05-22,2019-05-22 09:30:00,2019-06-21 09:30:00
VXM9,20190619,2019-06-19,2019-06-19 09:30:00,2019-07-19 09:30:00
VXN9,20190717,2019-07-17,2019-07-17 09:30:00,2019-08-16 09:30:00
VXQ9,20190821,2019-08-21,2019-08-21 09:30:00,2019-09-20 09:30:00


### Filling static values

In [587]:
data={}
data['front']={}
data['second']={}

for c in ['front','second']:
    data[c]['Symbol'] = cfg.loc[c].symbol
    data[c]['FWD_START_TIME'] = VIX.loc[data[c]['Symbol']].FWD_START_TIME
    data[c]['FWD_END_TIME'] = VIX.loc[data[c]['Symbol']].FWD_END_TIME
    data[c]['OVERRIDE_FORWARD_DATE']=cfg.loc[c,'OVERRIDE_FORWARD_DATE']
    data[c]['SPXFrontContract1'] = getSPXFrontContract(data[c]['FWD_START_TIME'],SPX,data[c]['OVERRIDE_FORWARD_DATE'],1)
    data[c]['SPXFrontContract2'] = getSPXFrontContract(data[c]['FWD_START_TIME'],SPX,data[c]['OVERRIDE_FORWARD_DATE'],2)
    data[c]['SPXBackContract1'] = getSPXBackContract(data[c]['FWD_END_TIME'],SPX,1)
    data[c]['SPXBackContract2'] = getSPXBackContract(data[c]['FWD_END_TIME'],SPX,2)
    
    data[c]['FRONT_FORWARD1_EXPIRATION'] = SPX.loc[data[c]['SPXFrontContract1'],'EXPIRATION']
    data[c]['FRONT_FORWARD2_EXPIRATION'] = SPX.loc[data[c]['SPXFrontContract2'],'EXPIRATION']
    data[c]['BACK_FORWARD1_EXPIRATION'] = SPX.loc[data[c]['SPXBackContract1'],'EXPIRATION']
    data[c]['BACK_FORWARD2_EXPIRATION'] = SPX.loc[data[c]['SPXBackContract2'],'EXPIRATION']
    
    display(data[c])

{'Symbol': 'VXF9',
 'FWD_START_TIME': Timestamp('2019-01-16 09:30:00'),
 'FWD_END_TIME': Timestamp('2019-02-15 09:30:00'),
 'OVERRIDE_FORWARD_DATE': 1.0,
 'SPXFrontContract1': 'SPH9',
 'SPXFrontContract2': 'SPH9',
 'SPXBackContract1': 'SPZ8',
 'SPXBackContract2': 'SPH9',
 'FRONT_FORWARD1_EXPIRATION': Timestamp('2019-03-14 00:00:00'),
 'FRONT_FORWARD2_EXPIRATION': Timestamp('2019-03-14 00:00:00'),
 'BACK_FORWARD1_EXPIRATION': Timestamp('2018-12-20 00:00:00'),
 'BACK_FORWARD2_EXPIRATION': Timestamp('2019-03-14 00:00:00')}

{'Symbol': 'VXG9',
 'FWD_START_TIME': Timestamp('2019-02-13 09:30:00'),
 'FWD_END_TIME': Timestamp('2019-03-15 09:30:00'),
 'OVERRIDE_FORWARD_DATE': 1.0,
 'SPXFrontContract1': 'SPH9',
 'SPXFrontContract2': 'SPH9',
 'SPXBackContract1': 'SPH9',
 'SPXBackContract2': 'SPM9',
 'FRONT_FORWARD1_EXPIRATION': Timestamp('2019-03-14 00:00:00'),
 'FRONT_FORWARD2_EXPIRATION': Timestamp('2019-03-14 00:00:00'),
 'BACK_FORWARD1_EXPIRATION': Timestamp('2019-03-14 00:00:00'),
 'BACK_FORWARD2_EXPIRATION': Timestamp('2019-06-20 00:00:00')}

In [264]:
#set up futures contracts
front = Future(exchange="CFE",symbol = "VIX",tradingClass = 'VX',secType = "FUT",localSymbol = cfg.loc['front'].symbol)        
second = Future(exchange="CFE",symbol = "VIX",tradingClass = 'VX',secType = "FUT",localSymbol = cfg.loc['second'].symbol)        
#spx = Future(exchange="GLOBEX",symbol = "SPX",tradingClass = 'SP',secType = "FUT",localSymbol = cfg.loc['sp'].symbol)  

#verifying contracts
ib.qualifyContracts(front)
ib.qualifyContracts(second)

#create data frame for prices
futuresPrices = pd.DataFrame(index = [front.localSymbol,second.localSymbol,spx.localSymbol],columns='bid ask'.split())

In [588]:
opt = Option(symbol='SPX', exchange='SMART',secType = "OPT")
chains = ib.reqContractDetails(opt)
options=util.df(chains)

In [589]:
options

,contract,marketName,minTick,orderTypes,validExchanges,priceMagnifier,underConId,longName,contractMonth,industry,...,callable,putable,coupon,convertible,maturity,issueDate,nextOptionDate,nextOptionType,nextOptionPartial,notes
0,"Option(conId=290075609, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
1,"Option(conId=290075612, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
2,"Option(conId=290075616, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
3,"Option(conId=290075621, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
4,"Option(conId=290075626, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
5,"Option(conId=290075631, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
6,"Option(conId=290075633, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
7,"Option(conId=290075636, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
8,"Option(conId=290075643, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,
9,"Option(conId=290075646, symbol='SPX', lastTrad...",SPX,0.05,"ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASK...","SMART,CBOE",1,416904,S&P 500 Stock Index,201901,Indices,...,False,False,0,False,,,,,False,


In [216]:
#ib.cancelMktData(contract)

In [217]:
#ib.disconnect()

In [590]:
#getting spx options
opt = Option(symbol='SPX', exchange='SMART',secType = "OPT",lastTradeDateOrContractMonth=cfg.loc['sp'].optionsExp)
chains = ib.reqContractDetails(opt)
options=util.df(chains)
strikes = sorted(set(options.contract.apply(lambda x:x.strike).values))
spxOptions = options.contract.values
spxOptionsSymbols = options.contract.apply(lambda x:x.localSymbol).values
spxOptionsPrices = pd.DataFrame(index = strikes,columns=['PUT BID','PUT ASK','CALL BID','CALL ASK'])

In [591]:
#getting front options
opt = Option(symbol='VIX', exchange='SMART',secType = "OPT",tradingClass='VIX',lastTradeDateOrContractMonth=cfg.loc['front'].optionsExp)
chains = ib.reqContractDetails(opt)
options=util.df(chains)
strikes = sorted(set(options.contract.apply(lambda x:x.strike).values))
frontOptions = options.contract.values
frontOptionsSymbols = options.contract.apply(lambda x:x.localSymbol).values
#create data frame for prices
frontOptionsPrices = pd.DataFrame(index = strikes,columns=['PUT BID','PUT ASK','CALL BID','CALL ASK'])

In [592]:
cfg.loc['front'].optionsExp

20190115

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
Error 10197, reqId 462: No market data during competing live session, contract: Option(conId=334828438, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2370.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02370000', tradingClass='SPX')
Error 10197, reqId 629: No market data during competing live session, contract: Option(conId=337284116, symbol='SPX', lastTradeDateOrContractMonth='20190117',

Error 10197, reqId 350: No market data during competing live session, contract: Option(conId=334827020, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2665.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02665000', tradingClass='SPX')
Error 10197, reqId 241: No market data during competing live session, contract: Option(conId=302976125, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2925.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02925000', tradingClass='SPX')
Error 10197, reqId 263: No market data during competing live session, contract: Option(conId=304784909, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3075.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P03075000', tradingClass='SPX')
Error 10197, reqId 682: No market data during competing live session, contract: Option(conId=346047556, symbol='SPX', 

Error 10197, reqId 229: No market data during competing live session, contract: Option(conId=290076186, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3100.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P03100000', tradingClass='SPX')
Error 10197, reqId 587: No market data during competing live session, contract: Option(conId=334830074, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3160.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P03160000', tradingClass='SPX')
Error 10197, reqId 371: No market data during competing live session, contract: Option(conId=334827351, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2795.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02795000', tradingClass='SPX')
Error 10197, reqId 204: No market data during competing live session, contract: Option(conId=290076086, symbol='SPX', 

Error 10197, reqId 182: No market data during competing live session, contract: Option(conId=290075997, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1725.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01725000', tradingClass='SPX')
Error 10197, reqId 697: No market data during competing live session, contract: Option(conId=346517447, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2055.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02055000', tradingClass='SPX')
Error 10197, reqId 126: No market data during competing live session, contract: Option(conId=290075774, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2200.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02200000', tradingClass='SPX')
Error 10197, reqId 711: No market data during competing live session, contract: Option(conId=346517599, symbol='SPX', 

...

Error 10197, reqId 18: No market data during competing live session, contract: Option(conId=326286366, symbol='VIX', lastTradeDateOrContractMonth='20190115', strike=19.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='VIX   190116C00019000', tradingClass='VIX')
Error 10197, reqId 589: No market data during competing live session, contract: Option(conId=334830098, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3180.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P03180000', tradingClass='SPX')
Error 10197, reqId 568: No market data during competing live session, contract: Option(conId=334829841, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3035.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P03035000', tradingClass='SPX')
Error 10197, reqId 186: No market data during competing live session, contract: Option(conId=290076014, symbol='SPX', las

Error 10197, reqId 601: No market data during competing live session, contract: Option(conId=336385816, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2020.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02020000', tradingClass='SPX')
Error 10197, reqId 379: No market data during competing live session, contract: Option(conId=334827446, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2845.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02845000', tradingClass='SPX')
Error 10197, reqId 476: No market data during competing live session, contract: Option(conId=334828593, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2460.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02460000', tradingClass='SPX')
Error 10197, reqId 212: No market data during competing live session, contract: Option(conId=290076117, symbol='SPX', 

Error 10197, reqId 161: No market data during competing live session, contract: Option(conId=290075912, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1150.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01150000', tradingClass='SPX')
Error 10197, reqId 206: No market data during competing live session, contract: Option(conId=290076092, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2325.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02325000', tradingClass='SPX')
Error 10197, reqId 333: No market data during competing live session, contract: Option(conId=334826806, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2560.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02560000', tradingClass='SPX')
Error 10197, reqId 304: No market data during competing live session, contract: Option(conId=334826429, symbol='SPX', 

Error 10197, reqId 181: No market data during competing live session, contract: Option(conId=290075992, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1700.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01700000', tradingClass='SPX')
Error 10197, reqId 557: No market data during competing live session, contract: Option(conId=334829589, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2965.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02965000', tradingClass='SPX')
Error 10197, reqId 362: No market data during competing live session, contract: Option(conId=334827191, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2740.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02740000', tradingClass='SPX')
Error 10197, reqId 668: No market data during competing live session, contract: Option(conId=345951617, symbol='SPX', 

Error 10197, reqId 383: No market data during competing live session, contract: Option(conId=334827497, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2870.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02870000', tradingClass='SPX')
Error 10197, reqId 576: No market data during competing live session, contract: Option(conId=334829949, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3085.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P03085000', tradingClass='SPX')
Error 10197, reqId 236: No market data during competing live session, contract: Option(conId=295209635, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2875.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02875000', tradingClass='SPX')
Error 10197, reqId 446: No market data during competing live session, contract: Option(conId=334828261, symbol='SPX', 

In [269]:
#getting second options
opt = Option(symbol='VIX', exchange='SMART',secType = "OPT",tradingClass='VIX',lastTradeDateOrContractMonth=cfg.loc['second'].optionsExp)
chains = ib.reqContractDetails(opt)
options=util.df(chains)
strikes = sorted(set(options.contract.apply(lambda x:x.strike).values))
secondOptions = options.contract.values
secondOptionsSymbols = options.contract.apply(lambda x:x.localSymbol).values
#create data frame for prices
secondOptionsPrices = pd.DataFrame(index = strikes,columns=['PUT BID','PUT ASK','CALL BID','CALL ASK'])

In [251]:
def onTicker(t):
    if True:
        row = t.contract.localSymbol
        strike = t.contract.strike
        right =  t.contract.right
        
        if row in [front.localSymbol,second.localSymbol,spx.localSymbol]:
            #print ('Futures')
            futuresPrices.loc[row] = (t.bid, t.ask)
        elif row in frontOptionsSymbols:
            #print ('FrontOption')
            if right=='P':
                frontOptionsPrices.loc[strike,['PUT BID','PUT ASK']] = (t.bid, t.ask)
            if right=='C':
                frontOptionsPrices.loc[strike,['CALL BID','CALL ASK']] = (t.bid, t.ask)
        elif row in spxOptionsSymbols:
            #print ('spxOption')
            if right=='P':
                spxOptionsPrices.loc[strike,['PUT BID','PUT ASK']] = (t.bid, t.ask)
            if right=='C':
                spxOptionsPrices.loc[strike,['CALL BID','CALL ASK']] = (t.bid, t.ask)
        elif row in secondOptionsSymbols:
            #print ('SecondOptions')
            if right=='P':
                secondOptionsPrices.loc[strike,['PUT BID','PUT ASK']] = (t.bid, t.ask)
            if right=='C':
                secondOptionsPrices.loc[strike,['CALL BID','CALL ASK']] = (t.bid, t.ask)

In [222]:
def onPrice(tickers):
    for t in tickers:
        onTicker(t)

In [252]:
ib.pendingTickersEvent += onPrice

In [253]:
#subscribe for futures market data
ticker = ib.reqMktData(front, '', False, False)
ticker = ib.reqMktData(second, '', False, False)
ticker = ib.reqMktData(spx, '', False, False)

In [254]:
#subscribe for marker data
for c in frontOptions:
    ticker = ib.reqMktData(c, '', False, False)
    time.sleep(1.0/60)

In [255]:
#subscribe for marker data
for c in secondOptions:
    ticker = ib.reqMktData(c, '', False, False)
    time.sleep(1.0/60)

Started to throttle requests
Stopped to throttle requests


In [258]:
#subscribe for marker data
for c in spxOptions:
    ticker = ib.reqMktData(c, '', False, False)
    time.sleep(1.0/60)

Started to throttle requests
Error 10090, reqId 1156: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075609, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1100.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C01100000', tradingClass='SPX')
Error 10090, reqId 1157: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075612, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1150.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C01150000', tradingClass='SPX')
Error 10090, reqId 1158: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is av

Error 10090, reqId 1181: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075711, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1800.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C01800000', tradingClass='SPX')
Error 10090, reqId 1182: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075714, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1825.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C01825000', tradingClass='SPX')
Error 10090, reqId 1183: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

Error 10090, reqId 1196: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075771, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2175.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02175000', tradingClass='SPX')
Error 10090, reqId 1197: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075774, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2200.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02200000', tradingClass='SPX')
Error 10090, reqId 1198: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

Error 10090, reqId 1225: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075885, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3100.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C03100000', tradingClass='SPX')
Error 10090, reqId 1216: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075850, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2675.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02675000', tradingClass='SPX')
Error 10090, reqId 1227: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

Error 10090, reqId 1243: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075956, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1475.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01475000', tradingClass='SPX')
Error 10090, reqId 1224: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=290075880, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=3000.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C03000000', tradingClass='SPX')
Error 10090, reqId 1235: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

...

Error 10090, reqId 1695: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=336948345, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2265.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02265000', tradingClass='SPX')
Error 10090, reqId 1697: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=336948355, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2295.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P02295000', tradingClass='SPX')
Error 10090, reqId 1696: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

Error 10090, reqId 1723: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=339681951, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2155.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02155000', tradingClass='SPX')
Error 10090, reqId 1726: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=339681976, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1880.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01880000', tradingClass='SPX')
Error 10090, reqId 1731: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

Error 10090, reqId 1739: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=345951617, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=900.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C00900000', tradingClass='SPX')
Error 10090, reqId 1738: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=345951614, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=850.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C00850000', tradingClass='SPX')
Error 10090, reqId 1741: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index

Error 10090, reqId 1753: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=346047556, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1075.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118P01075000', tradingClass='SPX')
Error 10090, reqId 1759: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=346517346, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1770.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C01770000', tradingClass='SPX')
Error 10090, reqId 1756: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

Error 10090, reqId 1761: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=346517369, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=1790.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C01790000', tradingClass='SPX')
Error 10090, reqId 1766: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=346517422, symbol='SPX', lastTradeDateOrContractMonth='20190117', strike=2035.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   190118C02035000', tradingClass='SPX')
Error 10090, reqId 1767: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Ind

In [278]:
cfg

,HARD_CODE_CUM_PROB,OBSERVED_CUM_PROB_RECENT,OVERRIDE_FORWARD_DATE,USE_OVERRIDE_VVIX,VVIX_OVERNIGHT_HARD_CODE,optionsExp,symbol
name,,,,,,,
second,0.98,0.9760,1.0,1.0,87.0,20190521,VXG9
front,0.85,0.8617,1.0,1.0,72.0,20190115,VXF9
sp,NaN,NaN,NaN,NaN,NaN,20190117,SPH9


In [399]:
def getSPXFrontContract(FWD_START_TIME,SPX,OVERRIDE_FORWARD_DATE,ind):
    if ind==1:
        if OVERRIDE_FORWARD_DATE==1:
            FT=SPX.loc[SPX.FWD_START_TIME>FWD_START_TIME,'FWD_START_TIME'].min()
        else:
            FT=SPX.loc[SPX.FWD_START_TIME<FWD_START_TIME,'FWD_START_TIME'].max()
    else:
        FT=SPX.loc[SPX.FWD_START_TIME>FWD_START_TIME,'FWD_START_TIME'].min()
        
    return SPX.loc[SPX.FWD_START_TIME==FT].index.values[0]

def getSPXBackContract(FWD_END_TIME,SPX,ind):
    FT=SPX.loc[SPX.FWD_START_TIME==FWD_END_TIME]
    
    if FT.shape[0]>0:
        return FT['SYMBOL'].values[0]
    
    if ind==1:
        FT=SPX.loc[SPX.FWD_START_TIME<FWD_END_TIME,'FWD_START_TIME'].max()
    else:
        FT=SPX.loc[SPX.FWD_START_TIME>FWD_END_TIME,'FWD_START_TIME'].min()
        
    return SPX.loc[SPX.FWD_START_TIME==FT].index.values[0]
def getTime2Expiration(CURRENT_TIME,T):
    return (T - CURRENT_TIME).total_seconds()/60/525600
def getKZero(Forward_Price,srtikesList):
    upList=([x for x in srtikesList if x>=Forward_Price])
    downList=([x for x in srtikesList if x<Forward_Price])
    if len(upList)>0:
        RoundUpPX = min(upList)
    else:
        RoundUpPX = np.nan
        print ("Error")

    if len(downList)>0:
        RoundDownPX = max(downList)
    else:
        RoundDownPX = np.nan
        print ("Error")
    RoundIncrement = RoundUpPX - RoundDownPX
    KZero =  int(Forward_Price / RoundIncrement)*RoundIncrement
    return KZero

In [460]:
def getVariance(options,CURRENT_TIME,Forward_Price,PUT_GAP_ALLOW,CALL_GAP_ALLOW,INT_RATE,FWD_START_TIME):
    KZero = getKZero(Forward_Price,options.index.values)
    T=getTime2Expiration(CURRENT_TIME,FWD_START_TIME)
    
    vixPuts=options[['PUT BID','PUT ASK']]
    vixPuts=vixPuts.sort_index(ascending=False)
    vixPuts['ValidStrike']=1
    vixPuts.loc[vixPuts.index>KZero,'ValidStrike']=0
    vixPuts.loc[vixPuts['PUT BID']<=0,'ValidStrike']=0
    vixPuts.loc[vixPuts['PUT ASK']<=0,'ValidStrike']=0
    vixPuts.loc[(vixPuts['ValidStrike'].rolling(PUT_GAP_ALLOW).sum()-vixPuts['ValidStrike']==0) & \
                (vixPuts.index!=KZero),'ValidStrike']=0   
    vixPuts=vixPuts.sort_index()
    vixPuts['STRIKE']=vixPuts.index
    
    vixCalls=options[['CALL BID','CALL ASK']]
    vixCalls=vixCalls.sort_index(ascending=True)
    vixCalls['ValidStrike']=1
    vixCalls.loc[vixCalls.index<KZero,'ValidStrike']=0
    vixCalls.loc[vixCalls['CALL BID']<=0,'ValidStrike']=0
    vixCalls.loc[vixCalls['CALL ASK']<=0,'ValidStrike']=0
    vixCalls.loc[(vixCalls['ValidStrike'].rolling(CALL_GAP_ALLOW).sum()-vixCalls['ValidStrike']==0) & \
                 (vixCalls.index!=KZero),'ValidStrike']=0 
    vixCalls['STRIKE']=vixCalls.index

    optionsData = vixPuts.merge(vixCalls, on='STRIKE',how='inner')
    optionsData.index=optionsData.STRIKE
    optionsData=optionsData.sort_index()
    optionsData['ValidStrike'] = (optionsData['ValidStrike_x']+optionsData['ValidStrike_y'])>0
    optionsData['StrikeInterval'] = 0.5*(optionsData.loc[optionsData['ValidStrike'],'STRIKE'].shift(-1)-optionsData.loc[optionsData['ValidStrike'],'STRIKE'].shift(1))
    
    tails=optionsData.loc[optionsData['ValidStrike'],'StrikeInterval'].index
    optionsData.loc[tails[0],'StrikeInterval'] = 0.5*(optionsData.loc[tails[1],'STRIKE'] - optionsData.loc[tails[0],'STRIKE'])
    optionsData.loc[tails[-1],'StrikeInterval'] = 0.5*(optionsData.loc[tails[-1],'STRIKE'] - optionsData.loc[tails[-2],'STRIKE'] )    

    optionsData['BID CONTRIB'] = optionsData['PUT BID']
    optionsData.loc[optionsData.STRIKE>KZero,'BID CONTRIB'] = optionsData['CALL BID']
    optionsData.loc[optionsData.STRIKE==KZero,'BID CONTRIB'] = 0.5*(optionsData['PUT BID']+optionsData['CALL BID'])
    
    optionsData['ASK CONTRIB'] = optionsData['PUT ASK']
    optionsData.loc[optionsData.STRIKE>KZero,'ASK CONTRIB'] = optionsData['CALL ASK']
    optionsData.loc[optionsData.STRIKE==KZero,'ASK CONTRIB'] = 0.5*(optionsData['PUT ASK']+optionsData['CALL ASK'])

    optionsData['BID CONTRIB']=optionsData['BID CONTRIB']*np.exp(INT_RATE*T)*optionsData['StrikeInterval']/pow(optionsData.STRIKE,2)
    optionsData['ASK CONTRIB']=optionsData['ASK CONTRIB']*np.exp(INT_RATE*T)*optionsData['StrikeInterval']/pow(optionsData.STRIKE,2)

    result = 0.5*(2/T*optionsData.loc[optionsData['ValidStrike'],'BID CONTRIB'].sum()*100+2/T*optionsData.loc[optionsData['ValidStrike'],'ASK CONTRIB'].sum()*100)
    
    return result

In [368]:
CURRENT_TIME = datetime.now()


In [585]:
SPXFrontContract1

'SPH9'

In [382]:
FRONT_FORWARD1_TIME = getTime2Expiration(CURRENT_TIME,FRONT_FORWARD1_EXPIRATION) 
FRONT_FORWARD2_TIME = getTime2Expiration(CURRENT_TIME,FRONT_FORWARD2_EXPIRATION) 
BACK_FORWARD1_TIME = getTime2Expiration(CURRENT_TIME,BACK_FORWARD1_EXPIRATION) 
BACK_FORWARD2_TIME = getTime2Expiration(CURRENT_TIME,BACK_FORWARD2_EXPIRATION) 

In [497]:
PUT_GAP_ALLOW = int(cfg.loc['front','PUT_GAP_ALLOW'])
CALL_GAP_ALLOW = int(cfg.loc['front','CALL_GAP_ALLOW'])
INT_RATE = 0.025

FRONT_FORWARD1_VARIANCE = getVariance(spxOptionsPrices,CURRENT_TIME,futuresPrices.loc[SPXFrontContract1].mean(),PUT_GAP_ALLOW,CALL_GAP_ALLOW,INT_RATE,SPX.loc[SPXFrontContract1].FWD_START_TIME)
if False:#SPXFrontContract1!=SPXFrontContract2:
    FRONT_FORWARD2_VARIANCE = getVariance(f,CURRENT_TIME,SPXFrontContract2,SPX)
else:
    FRONT_FORWARD2_VARIANCE = FRONT_FORWARD1_VARIANCE

BACK_FORWARD1_VARIANCE = 3#getVariance(f,CURRENT_TIME,SPXBackContract1,SPX)
BACK_FORWARD2_VARIANCE = 3#getVariance(f,CURRENT_TIME,SPXBackContract2,SPX)

In [487]:
VIXFrontContract = getVIXFrontContract('front',FWD_START_TIME,CURRENT_TIME)
VIXBackContract = getVIXBackContract('front',FWD_START_TIME,CURRENT_TIME)
    
FORWARD_START_TIME = getTime2Expiration(CURRENT_TIME,FWD_START_TIME)
FORWARD_END_TIME = getTime2Expiration(CURRENT_TIME,FWD_END_TIME)

In [482]:
def getVIXFrontContract(RANK,FWD_START_TIME,CURRENT_TIME):
    if RANK!='front':
        return cfg.loc[RANK].symbol
    if (FWD_START_TIME-CURRENT_TIME).days<7:
        return cfg.loc['second'].symbol
    return cfg.loc[RANK].symbol

In [486]:
def getVIXBackContract(RANK,FWD_START_TIME,CURRENT_TIME):
    if RANK!='front':
        return cfg.loc[RANK].symbol
    if (FWD_START_TIME-CURRENT_TIME).days<7:
        return cfg.loc['third'].symbol
    return cfg.loc['second'].symbol

In [498]:
FRONT_FORWARD_INTERP_VARIANCE = FRONT_FORWARD1_VARIANCE if SPXFrontContract1==SPXFrontContract2 else     (FRONT_FORWARD1_TIME*FRONT_FORWARD1_VARIANCE+((FRONT_FORWARD2_TIME*FRONT_FORWARD2_VARIANCE-FRONT_FORWARD1_TIME*FRONT_FORWARD1_VARIANCE)/(FRONT_FORWARD2_TIME-FRONT_FORWARD1_TIME))*(FORWARD_START_TIME-FRONT_FORWARD1_TIME))/FORWARD_START_TIME

BACK_FORWARD_INTERP_VARIANCE = BACK_FORWARD1_VARIANCE if SPXBackContract1==SPXBackContract2 else     (BACK_FORWARD1_TIME*BACK_FORWARD1_VARIANCE+((BACK_FORWARD2_TIME*BACK_FORWARD2_VARIANCE-BACK_FORWARD1_TIME*BACK_FORWARD1_VARIANCE)/(BACK_FORWARD2_TIME-BACK_FORWARD1_TIME))*(FORWARD_END_TIME-BACK_FORWARD1_TIME))/FORWARD_END_TIME

In [499]:
FORWARD_VOLATILITY = np.sqrt(((BACK_FORWARD_INTERP_VARIANCE*FORWARD_END_TIME-FRONT_FORWARD_INTERP_VARIANCE*FORWARD_START_TIME)/(FORWARD_END_TIME-FORWARD_START_TIME))/100)*100

In [506]:
FRONT_VARIANCE = getVariance(frontOptionsPrices,CURRENT_TIME,futuresPrices.loc[VIXFrontContract].mean(),PUT_GAP_ALLOW,CALL_GAP_ALLOW,INT_RATE,VIX.loc['front'].FWD_START_TIME)
if False:#VIXFrontContract!=VIXBackContract:
    BACK_VARIANCE = getVariance(f,CURRENT_TIME,VIXBackContract,VIX)
else:
    BACK_VARIANCE = FRONT_VARIANCE

In [513]:
FRONT_FORWARD_PRICE = futuresPrices.loc[VIXFrontContract].mean()
BACK_FORWARD_PRICE = futuresPrices.loc[VIXBackContract].mean()

In [516]:
KZERO_FRONT = getKZero(FRONT_FORWARD_PRICE,frontOptionsPrices.index)
KZERO_BACK = getKZero(BACK_FORWARD_PRICE,secondOptionsPrices.index)

In [520]:
FRONT_TIME = getTime2Expiration(CURRENT_TIME,VIX.loc['front','FWD_START_TIME']) 
BACK_TIME = getTime2Expiration(CURRENT_TIME,VIX.loc['second','FWD_START_TIME']) 
FRONT_TIME_HOURS = FRONT_TIME*525600
BACK_TIME_HOURS = BACK_TIME*525600

In [521]:
FRONT_ADJUSTED_VARIANCE = FRONT_VARIANCE/100-1/FRONT_TIME*pow(FRONT_FORWARD_PRICE/KZERO_FRONT-1,2)
BACK_ADJUSTED_VARIANCE = BACK_VARIANCE/100-1/BACK_TIME*pow(BACK_FORWARD_PRICE/KZERO_BACK-1,2)

FRONT_WEIGHT = 1 if VIXFrontContract==VIXBackContract else (BACK_TIME_HOURS-43200)/(BACK_TIME_HOURS-FRONT_TIME_HOURS)
BACK_WEIGHT = 1 if VIXFrontContract==VIXBackContract else (43200 - FRONT_TIME_HOURS)/(BACK_TIME_HOURS-FRONT_TIME_HOURS)

    

In [532]:
USE_OVERRIDE = cfg.loc['front','USE_OVERRIDE_VVIX']
VVIX_OVERNIGHT_HARD_CODE= cfg.loc['front','VVIX_OVERNIGHT_HARD_CODE']
HARD_CODE_CUM_PROB = cfg.loc['front','HARD_CODE_CUM_PROB']

VAR = FRONT_VARIANCE if VIXFrontContract==VIXBackContract else ((FRONT_TIME*FRONT_ADJUSTED_VARIANCE*FRONT_WEIGHT)+(BACK_TIME*BACK_ADJUSTED_VARIANCE*BACK_WEIGHT))*1200

VVIX_ADJUSTMENT = FORWARD_VOLATILITY*((1-np.sqrt(1-(VVIX_OVERNIGHT_HARD_CODE/100*FORWARD_START_TIME))) if USE_OVERRIDE==1 else (1-np.sqrt(1-(VAR/100*FORWARD_START_TIME))))

    

In [533]:
OUTRIGHT_MIDPOINT = futuresPrices.loc['VXF9'].mean()

IMPLIED_CUMULATIVE_PROBABILITY = min([norm.cdf((FORWARD_VOLATILITY-OUTRIGHT_MIDPOINT)/VVIX_ADJUSTMENT),0.999999])

CUMULATIVE_PROBABILITY_USED = HARD_CODE_CUM_PROB if USE_OVERRIDE else OBSERVED_CUM_PROB_RECENT

TV=FORWARD_VOLATILITY+VVIX_ADJUSTMENT*norm.isf(CUMULATIVE_PROBABILITY_USED)

In [534]:
TV

18.117010519904394